In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import time


import seaborn as sns
sns.set_style("whitegrid")

from scipy.spatial.distance import jensenshannon

import urllib3
import json

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Update data from nomis

In [2]:
df = pd.read_csv('../../data/testing/nomis/car_van.csv', index_col=0)

In [3]:
boroughs = ['City of London', 'Westminster', 'Kingston upon Thames', 'Bromley', 'Greenwich', 'Havering', 'Barnet', 'Enfield', 'Hillingdon', 'Brent']

In [4]:
dct = {}

for b in boroughs:
    dct[b] = df[b].values

In [5]:
df = pd.DataFrame(dct, index=df.index)

In [6]:
df.to_csv('../../data/testing_bias/nomis/car_van.csv')

# Compute values for synthetic data

In [7]:
boroughs = ['City of London', 'Westminster', 'Kingston upon Thames', 'Bromley', 'Greenwich', 'Havering', 'Barnet', 'Enfield', 'Hillingdon', 'Brent']
idx = ['Total', '0', '1', '2', '3', '4+', '#vehicles']

In [8]:
# ciDATGAN
dct = {}

for b in boroughs:
    df = pd.read_csv('../../data/ciDATGAN_bias/{}.csv'.format(b))

    vals = []
    vals.append(np.sum(1/df['hh_people']))
    for i in range(0, 4):
        vals.append(np.sum(1/df[df['hh_carvan'] == i]['hh_people']))
    vals.append(np.sum(1/df[df['hh_carvan'] >= 4]['hh_people']))

    vals.append(np.sum(df['hh_carvan']/df['hh_people']))

    dct[b] = vals

In [9]:
df = pd.DataFrame(dct, index=idx)
df.to_csv('../../data/testing_bias/ciDATGAN/car_van.csv')

In [10]:
# DATGAN
dct = {}

for b in boroughs:
    df = pd.read_csv('../../data/DATGAN_bias/{}.csv'.format(b))

    vals = []
    vals.append(np.sum(1/df['hh_people']))
    for i in range(0, 4):
        vals.append(np.sum(1/df[df['hh_carvan'] == i]['hh_people']))
    vals.append(np.sum(1/df[df['hh_carvan'] >= 4]['hh_people']))

    vals.append(np.sum(df['hh_carvan']/df['hh_people']))

    dct[b] = vals

In [11]:
df = pd.DataFrame(dct, index=idx)
df.to_csv('../../data/testing_bias/DATGAN/car_van.csv')

In [12]:
# oversample
dct = {}

for b in boroughs:
    df = pd.read_csv('../../data/oversample_bias/{}.csv'.format(b))

    vals = []
    vals.append(np.sum(1/df['hh_people']))
    for i in range(0, 4):
        vals.append(np.sum(1/df[df['hh_carvan'] == i]['hh_people']))
    vals.append(np.sum(1/df[df['hh_carvan'] >= 4]['hh_people']))

    vals.append(np.sum(df['hh_carvan']/df['hh_people']))

    dct[b] = vals

In [13]:
df = pd.DataFrame(dct, index=idx)
df.to_csv('../../data/testing_bias/oversample/car_van.csv')

# Compare

In [2]:
orig = pd.read_csv('../../data/testing_bias/nomis/car_van.csv', index_col=0)

In [3]:
files = [
    '../../data/testing_bias/ciDATGAN/car_van.csv',
    '../../data/testing_bias/DATGAN/car_van.csv',
    '../../data/testing_bias/oversample/car_van.csv'
]

dfs = [pd.read_csv(f, index_col=0) for f in files]
labels = ['ciDATGAN', 'DATGAN', 'Oversample LTDS']

In [4]:
orig = orig/orig.iloc[0]
orig = orig.iloc[1:-1]

errors = {}

for (df, l) in zip(dfs, labels):
    df = df/df.iloc[0]
    df = df.iloc[1:-1]

    errors[l] = []

    for c in orig.columns:
        errors[l].append(jensenshannon(df[c], orig[c]))

In [5]:
for l in labels:
    print('{}: {:.2e} \pm {:.2e}'.format(l, np.mean(errors[l]), np.std(errors[l])))

ciDATGAN: 1.23e-01 \pm 8.45e-02
DATGAN: 1.32e-01 \pm 7.17e-02
Oversample LTDS: 1.09e-01 \pm 3.84e-02
